<a href="https://colab.research.google.com/github/hammer1234567/hatErase/blob/master/dataset_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
import os
os.chdir('/content/drive/My Drive/ML Project/ML-PR/dataset')
!ls

Dataset_Exploration.ipynb  final_dataset.csv  hate_offensive  personal_attacks
detecting_insults	   hasoc	      jigsaw_toxic


In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import re 
import string
import matplotlib.pyplot as plt
%matplotlib inline

In [21]:
df = pd.read_csv('final_dataset.csv', encoding='latin-1')
print(df.count())
df.head()

text     190206
label    190206
dtype: int64


,text,label
0,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,0
1,@politico No. We should remember very clearly ...,1
2,@cricketworldcup Guess who would be the winner...,0
3,Corbyn is too politically intellectual for #Bo...,0
4,All the best to #TeamIndia for another swimmin...,0


In [18]:
df.groupby('label').describe()

text                                                                
        count  unique                                                top freq
label                                                                        
0      151100  151087  By wearing the #BalidaanBadge over his gloves ...    5
1       39106   39104  They heal, operate, diagnose  We consider them...    2